In [10]:
import numpy as np
import csv,random
from __future__ import division
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats as st
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from numpy.random import uniform
#load all the functions
#%run  myfunctions.ipynb
#%run  surrogate_agg.ipynb

In [1]:
def sur_error_rate(preds_matrix, eval_labels, P0=0.51, kappa=0.02):
#compute the error rates
#multi-testing for selecting the right solution for error rates

    label_matrix = np.zeros_like(preds_matrix, dtype=int)
    label_matrix[preds_matrix>=0.5] = 1
    label_matrix[preds_matrix<0.5] = 2
    label_matrix[preds_matrix<0] = -1
    
    #compute the error rates
    #multi-testing for selecting the right solution for error rates

    num_q = int(len(label_matrix))
    num_w = len(label_matrix[0])
    print(num_q,num_w)

    num_hypo = 11
    cnt_l = 0
    cnt_h = 0
    p0_l = []
    p1_l = []
    P0_est_l = []
    P1_post_l = []
    p0_h = []
    p1_h = []
    P0_est_h = []
    P1_post_h = []

    for t in range(num_hypo):
        kk = int(np.ceil(num_w*2))
        train_data = []
        train_data_Y = []
        for q in range(int(num_q/1)):
            ans_temp = label_matrix[q][:]
            list_taken = [] # extract the list of valid answers. 
            for ii in range(len(ans_temp)):
                if float(ans_temp[ii]) > 0: # -1 is regarded as not taking the task
                    list_taken.append(ii)
            #print list_taken
            
            data_temp = [float(ans_temp[x])-1 for x in list_taken]
            for k in range(kk):
                if len(data_temp)<3: continue
                data_temp2 = random.sample(data_temp,3)
                train_data.append(data_temp2)
        
        p0,p1,P0_est,ind = error_rate_priorfree(P0,train_data)
        print(p0,p1,ind)
        if (max(p0,p1) <= 1) & (min(p0,p1) > 0):
            if ind == 1:
                cnt_l = cnt_l + 1
                p0_l.append(p0)
                p1_l.append(p1)
                P0_est_l.append(P0_est)
                #P1_post_l.append(P1_post)
            else:
                cnt_h = cnt_h + 1
                p0_h.append(p0)
                p1_h.append(p1)
                P0_est_h.append(P0_est)
                #P1_post_h.append(P1_post)

    print(np.mean(p0_l),np.mean(p1_l))
    print(np.mean(p0_h),np.mean(p1_h))

    if cnt_l >= cnt_h:
        p0 = np.mean(p0_l)
        p1 = np.mean(p1_l)
        P0_est = np.mean(P0_est_l)
        #P1_post = np.mean(P1_post_l)
    else:
        p0 = np.mean(p0_h)
        p1 = np.mean(p1_h)
        P0_est = np.mean(P0_est_h)
        #P1_post = np.mean(P1_post_h)

        
    print('Estimated e0,e1,P0:~',p0,p1,P0_est)#,P1_post
    if np.isnan(p0) or np.isnan(p1) or np.isnan(P0_est):
        return np.NAN, np.NAN
    
    ###count in individual error rates
    err_ind = []
    for ww in range(len(eval_labels[0])):
        kk = 1000
        train_data = []
        ind_predict = []
        #q_list = [] #
        #for q in range(int(num_q/1)): #
        #    if float(label_matrix[q][ww]) > 0: #
        #        q_list.append(q) #
        q_list = [qid for qid in range(num_q)]
        
        for q in q_list:
            ans_temp = label_matrix[q][:]
            list_taken = [] # extract the list of valid answers. 
            for ii in range(len(ans_temp)):
                if float(ans_temp[ii]) > 0: # -1 is regarded as not taking the task
                    list_taken.append(ii)
            #print list_taken
            
            data_temp = [float(ans_temp[x])-1 for x in list_taken]
            for k in range(kk):
                ind_predict.append(float(eval_labels[q][ww])-1)
                data_temp2 = random.sample(data_temp,1)
                train_data.append(data_temp2[0])
   
        p0_m,p1_m = machine_error(P0_est,p0,p1,train_data, ind_predict)

        if abs(p0_m) <= 0.05:
            p0_m = 0
        if abs(p1_m) <= 0.05:
            p1_m = 0
            
        if abs(p0_m-1) <= 0.05:
            p0_m = 1
        if abs(p1_m-1) <= 0.05:
            p1_m = 1
            
        err_ind.append([p0_m,p1_m])
        
    for pp in err_ind:
        if pp[0]<-0.2 or pp[0]>1.2 or pp[1]<-0.2 or pp[1]>1.2:
            #pp[0], pp[1] = 0.5, 0.5
            continue
        else:
            pp[0] = max(min(pp[0],1-kappa),kappa)
            pp[1] = max(min(pp[1],1-kappa),kappa)
    return err_ind, P0_est
    

In [5]:
def surrogate_Brier(forecast, outcome, error0, error1):
    n = len(forecast)
    score = np.zeros(n)
    for i in range(n):
        p = forecast[i]
        e0 = error0[i]
        e1 = error1[i]
        if outcome[i] == 1:
            score[i] = ((1 - e0) * (2 * (1 - p)**2) - e1 * (2 * p**2)) / (1 - e1 - e0)
        else:
            score[i] = ((1 - e1) * (2 * p**2) - e0 * (2 * (1 - p)**2)) / (1 - e1 - e0)
    return score

In [ ]:
def estimate_sur_score(preds, agg_pred, agg_err):
    n, m = preds.shape

    score = np.NAN * np.ones(preds.shape)
    for j in range(m):
        curr_pred = preds[:, j]
        valid = curr_pred>=0.0
        curr_pred = curr_pred[valid]
        answerred_ifps_num = np.sum(valid)
        curr_score = np.zeros(answerred_ifps_num)
        e0, e1 = agg_err
        e1 = e1 * np.ones(answerred_ifps_num)
        e0 = e0 * np.ones(answerred_ifps_num)
        curr_score = (agg_pred[valid] * surrogate_Brier(curr_pred, np.ones(answerred_ifps_num), e0, e1) 
                      + (1-agg_pred[valid]) * surrogate_Brier(curr_pred, np.zeros(answerred_ifps_num), e0, e1))
        score[valid, j] = curr_score
        
    return score